In [1]:
import os
import glob
import json
import random
import numpy as np
import pandas as pd

In [2]:
info = json.load(open('meta_data/stage2/split_stage2.json'))

In [3]:
info

{'background': {'male': ['54',
   '51',
   '14',
   '03',
   '11',
   '22',
   '10',
   '40',
   '34',
   '45',
   '02',
   '17',
   '41',
   '55',
   '23',
   '05'],
  'female': ['26', '56', '59', '60']},
 'development': {'male': ['53',
   '21',
   '30',
   '44',
   '13',
   '20',
   '39',
   '09',
   '31',
   '01',
   '33',
   '35',
   '08',
   '18',
   '16',
   '46'],
  'female': ['58', '28', '47', '57']},
 'evaluation': {'male': ['50',
   '49',
   '27',
   '06',
   '15',
   '19',
   '48',
   '37',
   '42',
   '24',
   '04',
   '32',
   '38',
   '07',
   '29',
   '25'],
  'female': ['36', '43', '52', '12']}}

In [4]:
np.random.seed(2023)
random.seed(2023)

In [5]:
LOOP = 3
WORD = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
DIR = './data'

In [70]:
def prepare_data_stage2(root_dir,info,loop):
    pattern = 'loop%s/%s/%s/%s'
    for word in WORD:
        background = {
        'file': [],
        'speaker': [],
        'gender': [],
        'word': [], 
        }
        pairs = {
            'development': {
                'female':[],
                'male': [],
            },
            'evaluation': {
                'female':[],
                'male':[],
            }
        }
        folder_word = word + ('_'+word)*(loop-1)
        for phase in info:
            speakers = info[phase]
            for gender in speakers:
                if phase == 'background':
                    for spkr in speakers[gender]:
                        path = os.path.join(root_dir, 'loop%s'%(loop),folder_word , spkr)
                        for file in os.listdir(path):
                            background['file'].append(pattern %(loop, folder_word, spkr, file))
                            background['speaker'].append(spkr)
                            background['gender'].append(gender)
                            background['word'].append(folder_word)
                else:
                    for spkr in speakers[gender]:
                        imposters = set(info['development'][gender] + info['evaluation'][gender]) - set([spkr])
                        imposter_files = []
                        for imp in imposters:
                            imp_file = glob.glob(root_dir+('/loop%s/*/%s/*.wav')%(loop, imp))
                            imposter_files.extend(imp_file)

                        path = os.path.join(root_dir, 'loop%s'%(loop),folder_word , spkr)
                        files = os.listdir(path)
                        for file in files:
                            current_file = pattern % (loop, folder_word, spkr, file)
                            # Get target correct
                            # Accept pairs
                            other_files = [f for f in files if f != file]
                            accept_pairs = [['1', current_file, pattern % (loop, folder_word, spkr, o_file)] for o_file in other_files]

                            # Reject pairs
                            reject_pairs = []
                            n_pairs = len(files)
                            n_target_wrong = n_pairs // 2
                            n_imposters = n_pairs - n_target_wrong
                            
                            tw_all = glob.glob(root_dir+'/loop%s/[!%s]/%s/*.wav' % (loop, folder_word, spkr))
                            tw_set = random.choices(tw_all, k=n_target_wrong)
                            for tw_sample in tw_set:
                                reject_pairs.append(['0', current_file, tw_sample[len(root_dir)+1:]])

                            imposter_set = random.choices(imposter_files, k=n_imposters)
                            for imposter_sample in imposter_set:
                                reject_pairs.append(['0', current_file, imposter_sample[len(root_dir)+1:]])

                            
                            pairs[phase][gender].extend([accept_pairs] + [reject_pairs])
        break
        df_bg = pd.DataFrame(background)
        df_bg.to_csv(os.path.join(root_dir,'loop%s/%s/background.csv'%(loop,folder_word)), index=False)

        for phase in pairs:
            files = pairs[phase]
            for gender in files:
                fout = open(os.path.join(root_dir,'loop%s/%s/%s_%s.txt'%(loop,folder_word,gender,phase)), 'w')
                for list_file in files[gender]:
                    for file in list_file:
                        fout.write(" ".join(file) + '\n')

        


In [8]:
a = set([(1,1,1)])

In [13]:
a.add((1,1,2))

In [45]:
glob.glob('data/loop1/*/[!01]*/*.wav')

['data/loop1/0/39/0_15_.wav',
 'data/loop1/0/39/0_16_.wav',
 'data/loop1/0/39/0_39_.wav',
 'data/loop1/0/39/0_13_.wav',
 'data/loop1/0/39/0_20_.wav',
 'data/loop1/0/39/0_0_.wav',
 'data/loop1/0/39/0_47_.wav',
 'data/loop1/0/39/0_36_.wav',
 'data/loop1/0/39/0_31_.wav',
 'data/loop1/0/39/0_28_.wav',
 'data/loop1/0/39/0_2_.wav',
 'data/loop1/0/39/0_17_.wav',
 'data/loop1/0/39/0_37_.wav',
 'data/loop1/0/39/0_22_.wav',
 'data/loop1/0/39/0_6_.wav',
 'data/loop1/0/39/0_11_.wav',
 'data/loop1/0/39/0_30_.wav',
 'data/loop1/0/39/0_21_.wav',
 'data/loop1/0/39/0_3_.wav',
 'data/loop1/0/39/0_12_.wav',
 'data/loop1/0/39/0_10_.wav',
 'data/loop1/0/39/0_7_.wav',
 'data/loop1/0/39/0_14_.wav',
 'data/loop1/0/39/0_27_.wav',
 'data/loop1/0/39/0_4_.wav',
 'data/loop1/0/39/0_19_.wav',
 'data/loop1/0/39/0_8_.wav',
 'data/loop1/0/39/0_43_.wav',
 'data/loop1/0/39/0_26_.wav',
 'data/loop1/0/39/0_49_.wav',
 'data/loop1/0/39/0_18_.wav',
 'data/loop1/0/39/0_44_.wav',
 'data/loop1/0/39/0_33_.wav',
 'data/loop1/0/39

In [57]:
" ".join(["0", 'a', 'b']) + '\n'

'0 a b\n'